In [1]:
import csv, json, codecs, sys
import urllib.request
import urllib.error
import pandas as pd
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup
import datetime
from datetime import timezone

# Open-Meteo.com

# Координаты пастбища

In [4]:
latitude = 54.214106
longitude = 69.519512

# Параметры погоды

In [3]:
forecast_parameters = [
"temperature_2m_max",
"temperature_2m_min",
"apparent_temperature_max",
"apparent_temperature_min",
"precipitation_sum",
"rain_sum",
"showers_sum",
"snowfall_sum",
"precipitation_hours",
"precipitation_probability_max",
"precipitation_probability_min",
"precipitation_probability_mean",
"weathercode",
"sunrise",
"sunset",
"windspeed_10m_max",
"windgusts_10m_max",
"winddirection_10m_dominant",
"shortwave_radiation_sum",
"et0_fao_evapotranspiration",
"uv_index_max",
"uv_index_clear_sky_max",]

history_parameters = [
"temperature_2m_max",
"temperature_2m_min",
"apparent_temperature_max",
"apparent_temperature_min",
"precipitation_sum",
"rain_sum",
"snowfall_sum",
"precipitation_hours",
"weathercode",
"sunrise",
"sunset",
"windspeed_10m_max",
"windgusts_10m_max",
"winddirection_10m_dominant",
"shortwave_radiation_sum",
"et0_fao_evapotranspiration",]

def apply_params_to_URL(URL, parameters):
    URL += "&daily="
    for i, parameter in enumerate(parameters):
        if i < len(parameters) - 1:
            URL += parameter + ","
        else:
            URL += parameter
    URL += "&timezone=auto"
    
    print("URL updated: ", URL)
    return URL

def make_API_request(URL):
    try:
        # Convert from bytes to text
        resp_text = urllib.request.urlopen(URL).read().decode('UTF-8')
        # Use loads to decode from text
        json_obj = json.loads(resp_text)
        print("Successfull API request!")
        return json_obj
    except urllib.error.HTTPError  as e:
        ErrorInfo= e.read().decode()
        print('Error code: ', e.code, ErrorInfo)
        sys.exit()
    except  urllib.error.URLError as e:
        ErrorInfo= e.read().decode()
        print('Error code: ', e.code,ErrorInfo)
        sys.exit()
        

last_main_description = ""

def get_weather_mapping():
    global last_main_description
    
    url = "https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    weather_mapping = {}
    
    for i_table, table in enumerate(soup.select('table[BORDER]')):
        for i_row, row in enumerate(table.find_all('tr')[1:]):
            code_and_description = row.find_all('td')[:2]
            code = int(code_and_description[0].text.strip())
            description = code_and_description[1].text.strip()
            if not description.startswith("-"):
                last_main_description = description
            else:
                description = last_main_description
            weather_mapping[code] = description
            
            
    return weather_mapping

def weather_code_to_string(wmo_code):
    return weather_mapping.get(wmo_code, "Unknown Weather Code")

# Generate the weather mapping
weather_mapping = get_weather_mapping()

In [4]:
URL = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&past_days=2'

In [5]:
URL = apply_params_to_URL(URL, forecast_parameters)

URL updated:  https://api.open-meteo.com/v1/forecast?latitude=54.214106&longitude=69.519512&past_days=2&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,precipitation_probability_min,precipitation_probability_mean,weathercode,sunrise,sunset,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,uv_index_max,uv_index_clear_sky_max&timezone=auto


In [6]:
forecast_json_obj = make_API_request(URL)

Successfull API request!


# Единицы измерении значении параметров погоды

In [7]:
forecast_json_obj["daily_units"]

{'time': 'iso8601',
 'temperature_2m_max': '°C',
 'temperature_2m_min': '°C',
 'apparent_temperature_max': '°C',
 'apparent_temperature_min': '°C',
 'precipitation_sum': 'mm',
 'rain_sum': 'mm',
 'showers_sum': 'mm',
 'snowfall_sum': 'cm',
 'precipitation_hours': 'h',
 'precipitation_probability_max': '%',
 'precipitation_probability_min': '%',
 'precipitation_probability_mean': '%',
 'weathercode': 'wmo code',
 'sunrise': 'iso8601',
 'sunset': 'iso8601',
 'windspeed_10m_max': 'km/h',
 'windgusts_10m_max': 'km/h',
 'winddirection_10m_dominant': '°',
 'shortwave_radiation_sum': 'MJ/m²',
 'et0_fao_evapotranspiration': 'mm',
 'uv_index_max': '',
 'uv_index_clear_sky_max': ''}

In [8]:
forecast_json_obj["daily"]["time"]

['2024-01-10',
 '2024-01-11',
 '2024-01-12',
 '2024-01-13',
 '2024-01-14',
 '2024-01-15',
 '2024-01-16',
 '2024-01-17',
 '2024-01-18']

# weathercode значения:
https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM

# Предсказание погоды на 7 дней

In [9]:
forecast_df = pd.DataFrame(forecast_json_obj["daily"])

In [10]:
forecast_df

,time,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,...,weathercode,sunrise,sunset,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,uv_index_max,uv_index_clear_sky_max
0,2024-01-10,-14.9,-20.8,-19.3,-29.3,6.0,0.0,0.0,4.20,14.0,...,73,2024-01-10T09:38,2024-01-10T17:20,34.7,55.8,248,1.79,0.19,0.90,1.25
1,2024-01-11,-21.3,-26.8,-30.1,-33.9,0.0,0.0,0.0,0.00,0.0,...,3,2024-01-11T09:37,2024-01-11T17:21,33.1,52.6,246,3.68,0.15,1.25,1.25
2,2024-01-12,-13.2,-26.8,-20.6,-33.8,0.4,0.0,0.0,0.28,2.0,...,71,2024-01-12T09:36,2024-01-12T17:23,27.9,45.7,203,2.40,0.24,1.20,1.30
3,2024-01-13,-3.0,-12.4,-9.1,-19.6,2.3,0.0,0.0,1.61,11.0,...,71,2024-01-13T09:35,2024-01-13T17:24,25.2,41.0,192,1.59,0.25,1.20,1.30
4,2024-01-14,-6.3,-23.7,-11.5,-32.3,4.0,0.0,0.0,2.80,12.0,...,71,2024-01-14T09:35,2024-01-14T17:26,31.1,51.5,252,3.78,0.17,1.20,1.25
5,2024-01-15,-19.7,-24.4,-26.6,-32.1,0.0,0.0,0.0,0.00,0.0,...,3,2024-01-15T09:34,2024-01-15T17:27,26.9,44.6,238,3.82,0.17,1.35,1.35
6,2024-01-16,-18.2,-24.2,-23.7,-29.9,0.0,0.0,0.0,0.00,0.0,...,3,2024-01-16T09:33,2024-01-16T17:29,14.0,22.0,216,3.86,0.14,1.30,1.30
7,2024-01-17,-10.0,-18.7,-17.6,-24.6,0.0,0.0,0.0,0.00,0.0,...,3,2024-01-17T09:32,2024-01-17T17:31,30.0,49.3,210,2.64,0.43,1.20,1.25
8,2024-01-18,-10.4,-14.7,-16.3,-20.4,0.0,0.0,0.0,0.00,0.0,...,3,2024-01-18T09:30,2024-01-18T17:33,20.4,35.6,194,3.87,0.41,1.30,1.30


# История погоды по заданному диапазону

In [11]:
start_date = "2023-05-01" 
end_date = "2023-08-31"

In [12]:
Hist_URL = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}"

In [13]:
Hist_URL = apply_params_to_URL(Hist_URL, history_parameters)

URL updated:  https://archive-api.open-meteo.com/v1/archive?latitude=54.214106&longitude=69.519512&start_date=2023-05-01&end_date=2023-08-31&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weathercode,sunrise,sunset,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&timezone=auto


In [14]:
history_json_obj = make_API_request(Hist_URL)

Successfull API request!


In [15]:
history_df = pd.DataFrame(history_json_obj["daily"])

In [16]:
history_df

,time,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weathercode,sunrise,sunset,windspeed_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2023-05-01,13.1,1.4,8.3,-1.9,1.5,1.3,0.14,4.0,71,2023-05-01T05:45,2023-05-01T20:52,18.0,6,24.46,3.63
1,2023-05-02,14.5,-2.1,10.8,-6.3,0.0,0.0,0.00,0.0,1,2023-05-02T05:43,2023-05-02T20:54,11.5,77,24.57,3.68
2,2023-05-03,18.8,-0.4,15.0,-4.7,0.0,0.0,0.00,0.0,2,2023-05-03T05:41,2023-05-03T20:56,13.8,154,24.78,4.67
3,2023-05-04,22.1,5.5,18.1,1.0,0.0,0.0,0.00,0.0,0,2023-05-04T05:39,2023-05-04T20:58,16.9,231,24.41,5.72
4,2023-05-05,23.5,8.2,18.5,3.6,0.0,0.0,0.00,0.0,1,2023-05-05T05:37,2023-05-05T21:00,21.5,246,25.03,6.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-08-27,19.0,8.8,17.0,7.6,0.0,0.0,0.00,0.0,3,2023-08-27T06:20,2023-08-27T20:26,13.4,56,15.67,2.64
119,2023-08-28,19.1,12.2,16.7,11.1,0.8,0.8,0.00,5.0,51,2023-08-28T06:22,2023-08-28T20:24,13.8,314,11.85,2.26
120,2023-08-29,19.5,11.8,14.6,8.6,1.6,1.6,0.00,4.0,53,2023-08-29T06:23,2023-08-29T20:22,30.8,155,7.85,2.85
121,2023-08-30,18.2,12.5,16.1,10.0,8.0,8.0,0.00,14.0,61,2023-08-30T06:25,2023-08-30T20:19,27.6,159,10.78,2.23


In [17]:
history_df[history_df['time'].isin(['2023-05-04', '2023-05-05'])]

,time,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weathercode,sunrise,sunset,windspeed_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
3,2023-05-04,22.1,5.5,18.1,1.0,0.0,0.0,0.0,0.0,0,2023-05-04T05:39,2023-05-04T20:58,16.9,231,24.41,5.72
4,2023-05-05,23.5,8.2,18.5,3.6,0.0,0.0,0.0,0.0,1,2023-05-05T05:37,2023-05-05T21:00,21.5,246,25.03,6.22


# Open-Weather API

# Текущая погода

In [40]:
import requests

def get_history_weather_parameters(date):
    api_key = "99e5a57e01cae2c3bcbba5d43c31f645"  # Replace with your OpenWeatherMap API key
    date_object = datetime.datetime.strptime(date, "%Y-%m-%d")
    timestamp_utc = int(date_object.replace(tzinfo=timezone.utc).timestamp())
    url_template = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={}&lon={}&dt={}&appid={}"
    url = url_template.format(str(latitude), str(longitude), str(timestamp_utc), api_key)

    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_weather_df = pd.concat([json_normalize(data["data"])], axis=1).drop(columns=["weather", "dt", "sunrise", "sunset"])
        return current_weather_df
    else:
        print(f"Error: {response.status_code} - {response.text}")
    
def get_current_weather_parameters():
    api_key = "99e5a57e01cae2c3bcbba5d43c31f645"  # Replace with your OpenWeatherMap API key

#     url_template = "https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}"
    url_template = "https://api.openweathermap.org/data/3.0/onecall?lat={}&lon={}&appid={}&units=metric"
    
    url = url_template.format(str(latitude), str(longitude), api_key)
        
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        data["current"].pop('weather', None); current_weather_df = pd.DataFrame([data["current"]])
#         current_weather_df = pd.concat([json_normalize(data["main"]), json_normalize(data["wind"]), pd.DataFrame({'visibility': data["visibility"]}, index=[0])], axis=1)
        
        return current_weather_df
    else:
        print(f"Error: {response.status_code} - {response.text}")


In [15]:
# weather_df = get_current_weather_parameters()
weather_df = get_history_weather_parameters("2023-06-06")

In [16]:
weather_df

,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,293.36,292.79,1009,52,283.2,0,0.46,280


In [41]:
weather_df = get_current_weather_parameters()

In [42]:
weather_df

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust
0,1713719242,1713657963,1713710085,19.2,18.9,1024,66,12.7,0,36,10000,4.7,15,10.86
